신경망은 torch.nn 패키지를 사용하여 생성할 수 있음
nn은 모델을 정의하고, 미분하는데 autograd를 사용함.
nn.Module은 계층(layer)과 output을 반환하는 forward(input) 메서드를 포함

신경망의 일반적인 학습과정:
- 학습 가능한 매개변수(가중치)를 갖는 신경망을 정의
- 데이터셋 입력을 반복
- 입력을 신경망에서 전파
- 손실을 계산
- 기울기를 신경망의 매개변수들에 역으로 전파
- 신경망의 가중치를 갱신

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #입력 이미지 채널 1개, 출력채널 6개, 5x5의 정사각 컨볼루션 행렬
        self.conv1 = nn.Conv2d(1, 6, 5)
        #입력 이미지 채널 6개, 출력채널 16개, 5x5의 정사각 컨볼루션 행렬
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, input):
        # convolution layer c1 : 입력 이미지 채널 1, 출력 채널 6
        # 5x5 정사각 합성곱, 활성함수로 relu 사용
        # (N, 6, 28, 28)의 크기를 갖는 Tensor을 출력
        c1 = F.relu(self.conv1(input))
        # subsampling layer s2: 2x2 격자
        # (N, 6, 14, 14)의 크기를 갖는 Tensor을 출력
        s2 = F.max_pool2d(c1, (2,2))
        
        # convolution layer c3: 입력채널 6, 출력채널 16
        # 5x5 정사각 합성곱, 활성 함수로 relu 사용
        # (N, 16, 10, 10)의 크기를 갖는 Tensor을 출력
        c3 = F.relu(self.conv2(s2))
        # sumsampling layer s4: 2x2격자
        # (N, 16, 5, 5)의 크기를 갖는 Tensor을 출력
        s4 = F.max_pool2d(c3, 2)
        
        # 평탄화 연산 : (N, 16*5*5 = 400)의 텐서를 출력
        s4 = torch.flatten(s4, 1)
        
        # fully-connected layer f5
        # (N, 400) -> (N, 120)
        f5 = F.relu(self.fc1(s4))
        
        # fully-connected layer f6
        # (N, 120) -> (N, 84)
        f6 = F.relu(self.fc2(f5))
        
        # 가우시안 레이어 출력 (N, 84) -> (N, 10)
        output = self.fc3(f6)
        return output
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # 배치차원을 제외한 모든 차원을 하나로 평탄화
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [16]:
#모델의 학습 가능한 변수들은 net.parameters()에 의해 반환됨
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 weight

10
torch.Size([6, 1, 5, 5])


In [17]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0576,  0.0679,  0.0169,  0.1430,  0.0909,  0.0756, -0.0311, -0.0593,
         -0.0413,  0.0769]], grad_fn=<AddmmBackward0>)


In [18]:
net.zero_grad()
out.backward(torch.randn(1, 10))

torch.nn 은 미니배치(mini-batch)만 지원합니다. torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니배치만을 입력으로 받습니다.

예를 들어, nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.

만약 하나의 샘플만 있다면, input.unsqueeze(0) 을 사용해서 가상의 차원을 추가합니다.

In [19]:
output = net(input)
target = torch.randn(10) #예시를 위한 임의의 정답
target = target.view(1, -1) #출력과 같은 shape로 만듬
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7621, grad_fn=<MseLossBackward0>)


In [20]:
# loss를 backward하기 위해서는 loss.backward()만 해주면 됨
# 기존에 계산된 변화도의 값을 누적시키고 싶지 않다면 기존에 계산된 변화도를 0으로 만들어야함

net.zero_grad() #모든 매개변수의 변화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.gard after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.gard after backward
tensor([ 0.0056, -0.0020,  0.0112, -0.0086,  0.0047,  0.0049])


In [21]:
# 가중치 갱신
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad() #변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  #업데이트 진행


optimizer.zero_grad() 를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는 것에 유의하세요. 이는 역전파(Backprop) 섹션에서 설명한 것처럼 변화도가 누적되기 때문입니다.

In [ ]:
lr = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data * lr)